# Módulo 8: Vision Transformers e Mecanismos de Atenção

## 🎯 Objetivos de Aprendizagem

Ao final deste módulo, você será capaz de:

- ✅ Compreender os fundamentos dos mecanismos de atenção
- ✅ Entender a arquitetura Vision Transformer (ViT)
- ✅ Implementar modelos de atenção para visão computacional
- ✅ Analisar vantagens e limitações dos Vision Transformers
- ✅ Aplicar ViT em tarefas práticas de visão computacional

---

## 🎯 8.1 Introdução aos Mecanismos de Atenção

### Conceito Fundamental

**Mecanismos de Atenção** são componentes fundamentais que permitem aos modelos focar em partes específicas dos dados de entrada, revolucionando tanto o processamento de linguagem natural quanto a visão computacional.

![Introdução Mecanismos Atenção](https://github.com/rfapo/visao-computacional/blob/main/images/modulo8/introducao_mecanismos_atencao.png)

### Definição e Características

**Definição de Atenção:**
- **Foco seletivo**: Capacidade de concentrar recursos computacionais em partes relevantes
- **Pesos dinâmicos**: Atribuição de importância variável a diferentes elementos
- **Contexto global**: Consideração de todas as informações disponíveis
- **Processamento paralelo**: Capacidade de processar múltiplas relações simultaneamente

### Analogia Biológica

**Sistema de Atenção Humano:**
- **Atenção humana**: Foco seletivo em estímulos relevantes
- **Processamento visual**: Concentração em características importantes
- **Memória de trabalho**: Manutenção de informações relevantes
- **Decisão**: Integração de múltiplas fontes de informação

**Analogia com Redes Neurais:**
| Sistema Biológico | Rede Neural | Função |
|-------------------|-------------|--------|
| **Atenção seletiva** | Attention weights | Foco em features relevantes |
| **Processamento paralelo** | Multi-head attention | Múltiplas representações |
| **Contexto global** | Self-attention | Relações entre todos os elementos |
| **Memória de trabalho** | Hidden states | Manutenção de informação |

### Tipos de Atenção

![Tipos de Atenção](https://github.com/rfapo/visao-computacional/blob/main/images/modulo8/tipos_atencao.png)

#### **1. Self-Attention (Atenção Própria)**

**Características:**
- **Relações internas**: Cada elemento interage com todos os outros
- **Pesos computados**: Importância calculada dinamicamente
- **Contexto completo**: Consideração de toda a sequência
- **Paralelização**: Processamento simultâneo

**Fórmula:**
```
Attention(Q,K,V) = softmax(QK^T/√d_k)V
```

**Aplicações:**
- **Processamento de sequências**: NLP, time series
- **Análise de imagens**: Relações entre pixels
- **Recomendação**: Relações entre usuários e itens

#### **2. Multi-Head Attention**

**Características:**
- **Múltiplas cabeças**: Diferentes representações de atenção
- **Diversidade**: Captura diferentes tipos de relações
- **Concatenação**: Combinação de múltiplas cabeças
- **Flexibilidade**: Maior capacidade expressiva

**Fórmula:**
```
MultiHead(Q,K,V) = Concat(head_1, ..., head_h)W^O
head_i = Attention(QW_i^Q, KW_i^K, VW_i^V)
```

**Vantagens:**
- ✅ **Diversidade**: Diferentes tipos de atenção
- ✅ **Capacidade**: Maior poder expressivo
- ✅ **Robustez**: Menos sensível a falhas
- ✅ **Flexibilidade**: Adaptação a diferentes tarefas

#### **3. Cross-Attention**

**Características:**
- **Interação**: Entre sequências diferentes
- **Query**: De uma sequência
- **Key/Value**: De outra sequência
- **Aplicação**: Tradução, geração, multimodal

**Fórmula:**
```
CrossAttention(Q,K,V) = softmax(QK^T/√d_k)V
onde Q vem de uma sequência e K,V de outra
```

**Aplicações:**
- **Tradução**: Relações entre idiomas
- **Geração**: Relações entre texto e imagem
- **Multimodal**: Integração de diferentes modalidades

### Evolução dos Mecanismos de Atenção

![Evolução Mecanismos Atenção](https://github.com/rfapo/visao-computacional/blob/main/images/modulo8/evolucao_mecanismos_atencao.png)

#### **Progressão Histórica:**

| Ano | Marco | Contribuição |
|-----|-------|--------------|
| **2014** | Attention mechanism | Introdução para tradução |
| **2017** | Transformer | Self-attention puro |
| **2018** | BERT | Atenção bidirecional |
| **2020** | Vision Transformer | Atenção para imagens |
| **2021** | CLIP | Atenção multimodal |
| **2022** | DALL-E | Atenção para geração |

#### **Marcos Importantes:**
- **2014**: Neural Machine Translation by Jointly Learning to Align and Translate - Bahdanau et al.
- **2017**: Attention Is All You Need - Vaswani et al.
- **2020**: An Image is Worth 16x16 Words: Transformers for Image Recognition - Dosovitskiy et al.
- **2021**: CLIP: Learning Transferable Visual Representations - Radford et al.

---

## 🖼️ 8.2 Vision Transformers (ViT)

### Conceito Fundamental

**Vision Transformers** são arquiteturas que aplicam o mecanismo de atenção dos Transformers para processar imagens, tratando patches de imagem como "tokens" em uma sequência.

![Arquitetura Vision Transformer](https://github.com/rfapo/visao-computacional/blob/main/images/modulo8/arquitetura_vision_transformer.png)

### Componentes Principais

#### **1. Patch Embedding**

**Função:**
- **Divisão**: Imagem dividida em patches
- **Embedding**: Cada patch convertido em vetor
- **Linearização**: Patches tratados como sequência
- **Dimensionalidade**: Redução para dimensão fixa

**Processo:**
```
Imagem (H×W×C) → Patches (N×P²×C) → Embedding (N×D)
```

**Parâmetros:**
- **Patch size**: Tamanho de cada patch (ex: 16×16)
- **Embedding dimension**: Dimensão do vetor (ex: 768)
- **Number of patches**: N = (H×W)/(P²)

#### **2. Positional Embedding**

**Função:**
- **Posição**: Informação sobre localização dos patches
- **Soma**: Adicionada ao patch embedding
- **Aprendizado**: Parâmetros aprendíveis
- **Invariância**: Preserva informação espacial

**Tipos:**
- **1D**: Posição linear na sequência
- **2D**: Posição (x,y) na imagem
- **Aprendível**: Parâmetros otimizáveis
- **Sinusoidal**: Funções seno/cosseno

#### **3. Transformer Encoder**

**Estrutura:**
```
Multi-Head Self-Attention → Add & Norm → Feed Forward → Add & Norm
```

**Componentes:**
- **Multi-Head Attention**: Múltiplas cabeças de atenção
- **Feed Forward**: Rede feed-forward
- **Add & Norm**: Residual connection + layer normalization
- **Layers**: Múltiplas camadas empilhadas

#### **4. Classification Head**

**Função:**
- **CLS token**: Token especial para classificação
- **Pooling**: Agregação de informações
- **Linear**: Camada linear final
- **Softmax**: Probabilidades das classes

**Processo:**
```
CLS token → Transformer → CLS output → Linear → Softmax
```

### Vantagens dos Vision Transformers

#### **1. Escalabilidade**
- **Dados**: Performance melhora com mais dados
- **Modelo**: Arquitetura escalável
- **Computação**: Paralelização eficiente
- **Treinamento**: Estável com grandes datasets

#### **2. Interpretabilidade**
- **Attention maps**: Visualização de atenção
- **Patch importance**: Importância de cada patch
- **Global context**: Contexto global visível
- **Debugging**: Facilita debugging

#### **3. Flexibilidade**
- **Arquitetura**: Fácil modificação
- **Tarefas**: Adaptável a diferentes tarefas
- **Modalidades**: Extensível a outras modalidades
- **Integração**: Combinação com outras arquiteturas

### Limitações dos Vision Transformers

#### **1. Dados**
- **Requisito**: Necessita grandes datasets
- **Overfitting**: Risco com datasets pequenos
- **Transfer learning**: Dependência de modelos pré-treinados
- **Custo**: Treinamento caro

#### **2. Computação**
- **Complexidade**: O(n²) com número de patches
- **Memória**: Alto uso de memória
- **Inferência**: Pode ser lenta
- **Recursos**: Requer recursos computacionais

#### **3. Indução**
- **Bias**: Viés para padrões globais
- **Locais**: Menos eficiente para padrões locais
- **CNNs**: CNNs ainda melhores para algumas tarefas
- **Híbrido**: Combinação com CNNs pode ser melhor

---

## 🔍 8.3 Demonstração Prática: Mecanismos de Atenção

Vamos implementar e visualizar diferentes tipos de atenção:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

class AttentionMechanismsDemo:
    """Demonstração de diferentes mecanismos de atenção"""
    
    def __init__(self, seq_len=10, d_model=64):
        self.seq_len = seq_len
        self.d_model = d_model
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def create_sample_data(self):
        """Cria dados de exemplo para demonstração"""
        
        # Criar sequência de entrada
        x = torch.randn(1, self.seq_len, self.d_model)
        
        # Adicionar padrões específicos
        # Padrão 1: Valores altos no meio
        x[0, 3:7, :] += 2.0
        
        # Padrão 2: Valores baixos no início
        x[0, 0:2, :] -= 1.5
        
        # Padrão 3: Valores médios no final
        x[0, 8:, :] += 0.5
        
        return x
    
    def self_attention(self, x):
        """Implementa self-attention"""
        
        # Linear transformations
        W_q = nn.Linear(self.d_model, self.d_model, bias=False)
        W_k = nn.Linear(self.d_model, self.d_model, bias=False)
        W_v = nn.Linear(self.d_model, self.d_model, bias=False)
        
        # Compute Q, K, V
        Q = W_q(x)
        K = W_k(x)
        V = W_v(x)
        
        # Compute attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.d_model)
        attention_weights = F.softmax(scores, dim=-1)
        
        # Apply attention to values
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights
    
    def multi_head_attention(self, x, num_heads=8):
        """Implementa multi-head attention"""
        
        class MultiHeadAttention(nn.Module):
            def __init__(self, d_model, num_heads):
                super(MultiHeadAttention, self).__init__()
                self.d_model = d_model
                self.num_heads = num_heads
                self.d_k = d_model // num_heads
                
                self.W_q = nn.Linear(d_model, d_model)
                self.W_k = nn.Linear(d_model, d_model)
                self.W_v = nn.Linear(d_model, d_model)
                self.W_o = nn.Linear(d_model, d_model)
                
            def forward(self, x):
                batch_size, seq_len, d_model = x.size()
                
                # Linear transformations
                Q = self.W_q(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
                K = self.W_k(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
                V = self.W_v(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
                
                # Compute attention scores
                scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.d_k)
                attention_weights = F.softmax(scores, dim=-1)
                
                # Apply attention to values
                attended_values = torch.matmul(attention_weights, V)
                
                # Concatenate heads
                attended_values = attended_values.transpose(1, 2).contiguous().view(batch_size, seq_len, d_model)
                
                # Output projection
                output = self.W_o(attended_values)
                
                return output, attention_weights
        
        mha = MultiHeadAttention(self.d_model, num_heads)
        output, attention_weights = mha(x)
        
        return output, attention_weights
    
    def cross_attention(self, x1, x2):
        """Implementa cross-attention"""
        
        # Linear transformations
        W_q = nn.Linear(self.d_model, self.d_model, bias=False)
        W_k = nn.Linear(self.d_model, self.d_model, bias=False)
        W_v = nn.Linear(self.d_model, self.d_model, bias=False)
        
        # Compute Q from x1, K and V from x2
        Q = W_q(x1)
        K = W_k(x2)
        V = W_v(x2)
        
        # Compute attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.d_model)
        attention_weights = F.softmax(scores, dim=-1)
        
        # Apply attention to values
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights
    
    def visualize_attention_weights(self, attention_weights, title="Attention Weights"):
        """Visualiza pesos de atenção"""
        
        # Converter para numpy
        if isinstance(attention_weights, torch.Tensor):
            attention_weights = attention_weights.detach().numpy()
        
        # Remover dimensões de batch se necessário
        if attention_weights.ndim == 4:  # Multi-head attention
            attention_weights = attention_weights[0]  # Primeiro batch
        elif attention_weights.ndim == 3:
            attention_weights = attention_weights[0]  # Primeiro batch
        
        # Visualizar
        plt.figure(figsize=(10, 8))
        
        if attention_weights.ndim == 3:  # Multi-head
            # Visualizar cada cabeça
            num_heads = attention_weights.shape[0]
            cols = min(4, num_heads)
            rows = (num_heads + cols - 1) // cols
            
            for i in range(num_heads):
                plt.subplot(rows, cols, i + 1)
                sns.heatmap(attention_weights[i], cmap='Blues', cbar=True)
                plt.title(f'Head {i + 1}')
                plt.xlabel('Key Position')
                plt.ylabel('Query Position')
        else:  # Single head
            sns.heatmap(attention_weights, cmap='Blues', cbar=True)
            plt.title(title)
            plt.xlabel('Key Position')
            plt.ylabel('Query Position')
        
        plt.tight_layout()
        plt.show()
    
    def demonstrate_attention_mechanisms(self):
        """Demonstra diferentes mecanismos de atenção"""
        
        # Criar dados de exemplo
        x = self.create_sample_data()
        
        print("=== DEMONSTRAÇÃO: MECANISMOS DE ATENÇÃO ===")
        print(f"Sequência de entrada: {x.shape}")
        print(f"Dimensão do modelo: {self.d_model}")
        print(f"Comprimento da sequência: {self.seq_len}")
        
        # Self-attention
        print("\n1. Self-Attention:")
        sa_output, sa_weights = self.self_attention(x)
        print(f"   Output shape: {sa_output.shape}")
        print(f"   Attention weights shape: {sa_weights.shape}")
        
        # Multi-head attention
        print("\n2. Multi-Head Attention:")
        mha_output, mha_weights = self.multi_head_attention(x, num_heads=8)
        print(f"   Output shape: {mha_output.shape}")
        print(f"   Attention weights shape: {mha_weights.shape}")
        
        # Cross-attention
        print("\n3. Cross-Attention:")
        x2 = torch.randn(1, self.seq_len, self.d_model)  # Segunda sequência
        ca_output, ca_weights = self.cross_attention(x, x2)
        print(f"   Output shape: {ca_output.shape}")
        print(f"   Attention weights shape: {ca_weights.shape}")
        
        # Visualizar pesos de atenção
        print("\n=== VISUALIZAÇÃO DOS PESOS DE ATENÇÃO ===")
        
        # Self-attention
        self.visualize_attention_weights(sa_weights, "Self-Attention Weights")
        
        # Multi-head attention
        self.visualize_attention_weights(mha_weights, "Multi-Head Attention Weights")
        
        # Cross-attention
        self.visualize_attention_weights(ca_weights, "Cross-Attention Weights")
        
        # Análise quantitativa
        print("\n=== ANÁLISE QUANTITATIVA ===")
        
        # Self-attention
        sa_weights_np = sa_weights.detach().numpy()[0]
        print(f"\nSelf-Attention:")
        print(f"  - Peso máximo: {np.max(sa_weights_np):.4f}")
        print(f"  - Peso mínimo: {np.min(sa_weights_np):.4f}")
        print(f"  - Peso médio: {np.mean(sa_weights_np):.4f}")
        print(f"  - Entropia: {np.sum(-sa_weights_np * np.log(sa_weights_np + 1e-8)):.4f}")
        
        # Multi-head attention
        mha_weights_np = mha_weights.detach().numpy()[0]
        print(f"\nMulti-Head Attention:")
        print(f"  - Peso máximo: {np.max(mha_weights_np):.4f}")
        print(f"  - Peso mínimo: {np.min(mha_weights_np):.4f}")
        print(f"  - Peso médio: {np.mean(mha_weights_np):.4f}")
        print(f"  - Entropia média: {np.mean([np.sum(-head * np.log(head + 1e-8)) for head in mha_weights_np]):.4f}")
        
        # Cross-attention
        ca_weights_np = ca_weights.detach().numpy()[0]
        print(f"\nCross-Attention:")
        print(f"  - Peso máximo: {np.max(ca_weights_np):.4f}")
        print(f"  - Peso mínimo: {np.min(ca_weights_np):.4f}")
        print(f"  - Peso médio: {np.mean(ca_weights_np):.4f}")
        print(f"  - Entropia: {np.sum(-ca_weights_np * np.log(ca_weights_np + 1e-8)):.4f}")
        
        return {
            'self_attention': (sa_output, sa_weights),
            'multi_head_attention': (mha_output, mha_weights),
            'cross_attention': (ca_output, ca_weights)
        }

# Executar demonstração
print("=== DEMONSTRAÇÃO: MECANISMOS DE ATENÇÃO ===")
attention_demo = AttentionMechanismsDemo(seq_len=10, d_model=64)
results = attention_demo.demonstrate_attention_mechanisms()

### Análise dos Resultados

**Observações Importantes:**

1. **Self-Attention**:
   - **Pesos**: Distribuição de atenção entre elementos
   - **Padrões**: Identificação de relações importantes
   - **Contexto**: Consideração de toda a sequência

2. **Multi-Head Attention**:
   - **Diversidade**: Diferentes cabeças capturam diferentes padrões
   - **Robustez**: Menos sensível a falhas
   - **Capacidade**: Maior poder expressivo

3. **Cross-Attention**:
   - **Interação**: Relações entre sequências diferentes
   - **Flexibilidade**: Adaptação a diferentes modalidades
   - **Aplicação**: Útil para tarefas multimodais

---

## 🖼️ 8.4 Demonstração Prática: Vision Transformer Simples

Vamos implementar e visualizar um Vision Transformer simples:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import cv2

class SimpleVisionTransformer:
    """Implementação de um Vision Transformer simples"""
    
    def __init__(self, img_size=32, patch_size=8, num_classes=10, d_model=128, num_heads=8, num_layers=6):
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_classes = num_classes
        self.d_model = d_model
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Calcular número de patches
        self.num_patches = (img_size // patch_size) ** 2
        
        # Inicializar modelo
        self.vit = self._build_vit()
        
    def _build_vit(self):
        """Constrói o Vision Transformer"""
        
        class PatchEmbedding(nn.Module):
            def __init__(self, img_size, patch_size, d_model):
                super(PatchEmbedding, self).__init__()
                self.img_size = img_size
                self.patch_size = patch_size
                self.num_patches = (img_size // patch_size) ** 2
                
                # Patch embedding
                self.patch_embed = nn.Conv2d(3, d_model, kernel_size=patch_size, stride=patch_size)
                
                # Positional embedding
                self.pos_embed = nn.Parameter(torch.randn(1, self.num_patches + 1, d_model))
                
                # CLS token
                self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))
                
            def forward(self, x):
                B = x.shape[0]
                
                # Patch embedding
                x = self.patch_embed(x)  # (B, d_model, H/patch_size, W/patch_size)
                x = x.flatten(2).transpose(1, 2)  # (B, num_patches, d_model)
                
                # Adicionar CLS token
                cls_tokens = self.cls_token.expand(B, -1, -1)
                x = torch.cat((cls_tokens, x), dim=1)  # (B, num_patches + 1, d_model)
                
                # Adicionar positional embedding
                x = x + self.pos_embed
                
                return x
        
        class TransformerBlock(nn.Module):
            def __init__(self, d_model, num_heads):
                super(TransformerBlock, self).__init__()
                
                # Multi-head attention
                self.attention = nn.MultiheadAttention(d_model, num_heads, batch_first=True)
                
                # Feed forward
                self.feed_forward = nn.Sequential(
                    nn.Linear(d_model, d_model * 4),
                    nn.GELU(),
                    nn.Linear(d_model * 4, d_model)
                )
                
                # Layer normalization
                self.norm1 = nn.LayerNorm(d_model)
                self.norm2 = nn.LayerNorm(d_model)
                
            def forward(self, x):
                # Self-attention
                attn_output, attn_weights = self.attention(x, x, x)
                x = x + attn_output
                x = self.norm1(x)
                
                # Feed forward
                ff_output = self.feed_forward(x)
                x = x + ff_output
                x = self.norm2(x)
                
                return x, attn_weights
        
        class VisionTransformer(nn.Module):
            def __init__(self, img_size, patch_size, num_classes, d_model, num_heads, num_layers):
                super(VisionTransformer, self).__init__()
                
                # Patch embedding
                self.patch_embedding = PatchEmbedding(img_size, patch_size, d_model)
                
                # Transformer blocks
                self.transformer_blocks = nn.ModuleList([
                    TransformerBlock(d_model, num_heads) for _ in range(num_layers)
                ])
                
                # Classification head
                self.classifier = nn.Linear(d_model, num_classes)
                
            def forward(self, x):
                # Patch embedding
                x = self.patch_embedding(x)
                
                # Transformer blocks
                attention_weights = []
                for transformer_block in self.transformer_blocks:
                    x, attn_weights = transformer_block(x)
                    attention_weights.append(attn_weights)
                
                # Classification
                cls_output = x[:, 0]  # CLS token
                logits = self.classifier(cls_output)
                
                return logits, attention_weights
        
        return VisionTransformer(
            self.img_size, self.patch_size, self.num_classes,
            self.d_model, self.num_heads, self.num_layers
        ).to(self.device)
    
    def create_sample_images(self, num_samples=16):
        """Cria imagens de exemplo para demonstração"""
        
        images = []
        
        for _ in range(num_samples):
            # Criar imagem com padrões simples
            img = np.random.rand(3, self.img_size, self.img_size) * 2 - 1  # Normalizar para [-1, 1]
            
            # Adicionar padrões
            if np.random.random() > 0.5:
                # Padrão circular
                center_x, center_y = np.random.randint(8, self.img_size-8, 2)
                radius = np.random.randint(4, 8)
                
                y, x = np.ogrid[:self.img_size, :self.img_size]
                mask = (x - center_x)**2 + (y - center_y)**2 <= radius**2
                
                img[0, mask] = 1.0  # Red
                img[1, mask] = 0.0  # Green
                img[2, mask] = 0.0  # Blue
            else:
                # Padrão retangular
                x1, y1 = np.random.randint(0, self.img_size-12, 2)
                x2, y2 = x1 + np.random.randint(8, 16), y1 + np.random.randint(8, 16)
                
                img[0, y1:y2, x1:x2] = 0.0  # Red
                img[1, y1:y2, x1:x2] = 1.0  # Green
                img[2, y1:y2, x1:x2] = 0.0  # Blue
            
            images.append(img)
        
        return torch.FloatTensor(images)
    
    def visualize_patches(self, img, patch_size):
        """Visualiza patches de uma imagem"""
        
        # Converter para numpy
        if isinstance(img, torch.Tensor):
            img = img.detach().numpy()
        
        # Normalizar para [0, 1]
        img = (img + 1) / 2
        img = np.clip(img, 0, 1)
        
        # Converter para formato de visualização
        img_vis = img.transpose(1, 2, 0)
        
        # Criar grid de patches
        h, w = img_vis.shape[:2]
        num_patches_h = h // patch_size
        num_patches_w = w // patch_size
        
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))
        ax.imshow(img_vis)
        
        # Desenhar linhas dos patches
        for i in range(num_patches_h + 1):
            ax.axhline(y=i * patch_size, color='red', linewidth=1)
        for j in range(num_patches_w + 1):
            ax.axvline(x=j * patch_size, color='red', linewidth=1)
        
        ax.set_title(f'Imagem com Patches ({patch_size}×{patch_size})')
        ax.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        return num_patches_h * num_patches_w
    
    def visualize_attention_maps(self, attention_weights, img_size, patch_size):
        """Visualiza mapas de atenção"""
        
        # Converter para numpy
        if isinstance(attention_weights, torch.Tensor):
            attention_weights = attention_weights.detach().numpy()
        
        # Remover dimensões de batch
        if attention_weights.ndim == 4:  # Multi-head attention
            attention_weights = attention_weights[0]  # Primeiro batch
        elif attention_weights.ndim == 3:
            attention_weights = attention_weights[0]  # Primeiro batch
        
        # Calcular número de patches
        num_patches = (img_size // patch_size) ** 2
        
        # Visualizar
        if attention_weights.ndim == 3:  # Multi-head
            num_heads = attention_weights.shape[0]
            cols = min(4, num_heads)
            rows = (num_heads + cols - 1) // cols
            
            fig, axes = plt.subplots(rows, cols, figsize=(15, 4 * rows))
            if rows == 1:
                axes = [axes] if cols == 1 else axes
            else:
                axes = axes.flatten()
            
            for i in range(num_heads):
                # CLS token attention (primeira linha)
                cls_attention = attention_weights[i, 0, 1:]  # Remover CLS token
                
                # Reshape para imagem
                patch_size_h = img_size // patch_size
                attention_map = cls_attention.reshape(patch_size_h, patch_size_h)
                
                # Visualizar
                im = axes[i].imshow(attention_map, cmap='hot', interpolation='nearest')
                axes[i].set_title(f'Head {i + 1} - CLS Attention')
                axes[i].axis('off')
                plt.colorbar(im, ax=axes[i])
            
            # Ocultar eixos extras
            for i in range(num_heads, len(axes)):
                axes[i].axis('off')
        else:  # Single head
            # CLS token attention
            cls_attention = attention_weights[0, 1:]  # Remover CLS token
            
            # Reshape para imagem
            patch_size_h = img_size // patch_size
            attention_map = cls_attention.reshape(patch_size_h, patch_size_h)
            
            # Visualizar
            plt.figure(figsize=(8, 8))
            plt.imshow(attention_map, cmap='hot', interpolation='nearest')
            plt.title('CLS Token Attention Map')
            plt.colorbar()
            plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    def demonstrate_vit(self):
        """Demonstra o Vision Transformer"""
        
        print("=== DEMONSTRAÇÃO: VISION TRANSFORMER ===")
        print(f"Tamanho da imagem: {self.img_size}×{self.img_size}")
        print(f"Tamanho do patch: {self.patch_size}×{self.patch_size}")
        print(f"Número de patches: {self.num_patches}")
        print(f"Dimensão do modelo: {self.d_model}")
        print(f"Número de cabeças: {self.num_heads}")
        print(f"Número de camadas: {self.num_layers}")
        
        # Criar imagens de exemplo
        images = self.create_sample_images(16)
        
        # Visualizar patches
        print("\n=== VISUALIZAÇÃO DE PATCHES ===")
        num_patches = self.visualize_patches(images[0], self.patch_size)
        print(f"Número de patches por imagem: {num_patches}")
        
        # Processar com ViT
        print("\n=== PROCESSAMENTO COM VIT ===")
        self.vit.eval()
        
        with torch.no_grad():
            # Processar primeira imagem
            img = images[0:1].to(self.device)
            logits, attention_weights = self.vit(img)
            
            print(f"Logits shape: {logits.shape}")
            print(f"Número de camadas de atenção: {len(attention_weights)}")
            print(f"Shape dos pesos de atenção: {attention_weights[0].shape}")
            
            # Predição
            probs = F.softmax(logits, dim=1)
            predicted_class = torch.argmax(probs, dim=1).item()
            confidence = probs[0, predicted_class].item()
            
            print(f"Classe predita: {predicted_class}")
            print(f"Confiança: {confidence:.4f}")
        
        # Visualizar mapas de atenção
        print("\n=== VISUALIZAÇÃO DE MAPAS DE ATENÇÃO ===")
        
        # Primeira camada
        print("\nPrimeira camada:")
        self.visualize_attention_maps(attention_weights[0], self.img_size, self.patch_size)
        
        # Última camada
        print("\nÚltima camada:")
        self.visualize_attention_maps(attention_weights[-1], self.img_size, self.patch_size)
        
        # Análise quantitativa
        print("\n=== ANÁLISE QUANTITATIVA ===")
        
        # Analisar evolução da atenção
        first_layer_attention = attention_weights[0].detach().numpy()[0]  # Primeira cabeça
        last_layer_attention = attention_weights[-1].detach().numpy()[0]  # Primeira cabeça
        
        print(f"\nEvolução da Atenção:")
        print(f"  - Primeira camada - Entropia: {np.sum(-first_layer_attention * np.log(first_layer_attention + 1e-8)):.4f}")
        print(f"  - Última camada - Entropia: {np.sum(-last_layer_attention * np.log(last_layer_attention + 1e-8)):.4f}")
        print(f"  - Diferença de entropia: {np.sum(-last_layer_attention * np.log(last_layer_attention + 1e-8)) - np.sum(-first_layer_attention * np.log(first_layer_attention + 1e-8)):.4f}")
        
        # Analisar concentração da atenção
        first_cls_attention = first_layer_attention[0, 1:]  # CLS token
        last_cls_attention = last_layer_attention[0, 1:]  # CLS token
        
        print(f"\nConcentração da Atenção (CLS Token):")
        print(f"  - Primeira camada - Max: {np.max(first_cls_attention):.4f}")
        print(f"  - Primeira camada - Min: {np.min(first_cls_attention):.4f}")
        print(f"  - Última camada - Max: {np.max(last_cls_attention):.4f}")
        print(f"  - Última camada - Min: {np.min(last_cls_attention):.4f}")
        
        return {
            'logits': logits,
            'attention_weights': attention_weights,
            'predicted_class': predicted_class,
            'confidence': confidence
        }

# Executar demonstração
print("=== DEMONSTRAÇÃO: VISION TRANSFORMER ===")
vit_demo = SimpleVisionTransformer(
    img_size=32, patch_size=8, num_classes=10,
    d_model=128, num_heads=8, num_layers=6
)
results = vit_demo.demonstrate_vit()

### Análise dos Resultados

**Observações Importantes:**

1. **Patches**:
   - **Divisão**: Imagem dividida em patches regulares
   - **Embedding**: Cada patch convertido em vetor
   - **Posição**: Informação de posição preservada

2. **Mapas de Atenção**:
   - **Evolução**: Atenção muda entre camadas
   - **Concentração**: Última camada mais concentrada
   - **Interpretabilidade**: Visualização de foco

3. **Classificação**:
   - **CLS Token**: Agrega informação global
   - **Confiança**: Medida de certeza da predição
   - **Performance**: ViT funciona bem para classificação

---

## 📊 8.5 Comparação: CNNs vs Vision Transformers

### Análise Comparativa

![Comparação CNNs vs ViT](https://github.com/rfapo/visao-computacional/blob/main/images/modulo8/comparacao_cnns_vit.png)

#### **Arquitetura**

| Aspecto | CNNs | Vision Transformers |
|---------|------|---------------------|
| **Indução** | Local | Global |
| **Paralelização** | Limitada | Completa |
| **Escalabilidade** | Moderada | Alta |
| **Interpretabilidade** | Baixa | Alta |

#### **Performance**

| Aspecto | CNNs | Vision Transformers |
|---------|------|---------------------|
| **Dados pequenos** | Boa | Limitada |
| **Dados grandes** | Boa | Excelente |
| **Treinamento** | Estável | Estável |
| **Inferência** | Rápida | Moderada |

#### **Aplicações**

| Aspecto | CNNs | Vision Transformers |
|---------|------|---------------------|
| **Classificação** | Excelente | Excelente |
| **Detecção** | Excelente | Boa |
| **Segmentação** | Excelente | Boa |
| **Transfer Learning** | Boa | Excelente |

### Quando Usar Cada Um

#### **Use CNNs quando:**
- ✅ **Dados limitados** estão disponíveis
- ✅ **Padrões locais** são importantes
- ✅ **Velocidade** é prioritária
- ✅ **Recursos limitados** estão disponíveis

#### **Use Vision Transformers quando:**
- ✅ **Grandes datasets** estão disponíveis
- ✅ **Padrões globais** são importantes
- ✅ **Interpretabilidade** é necessária
- ✅ **Transfer learning** é prioritário

---

## 📝 Resumo do Módulo 8

### Principais Conceitos Abordados

1. **Fundamentos**: Mecanismos de atenção
2. **Tipos**: Self-attention, Multi-head, Cross-attention
3. **Vision Transformers**: Arquitetura para imagens
4. **Implementação**: Demonstrações práticas
5. **Comparação**: CNNs vs Vision Transformers

### Demonstrações Práticas

**1. Mecanismos de Atenção:**
   - Implementação de diferentes tipos de atenção
   - Visualização de pesos de atenção
   - Análise quantitativa

**2. Vision Transformer:**
   - Implementação de ViT simples
   - Visualização de patches e mapas de atenção
   - Análise de evolução da atenção

### Próximos Passos

No **Módulo 9**, exploraremos **Foundation Models** para visão computacional, incluindo CLIP, DALL-E e GPT-4V.

### Referências Principais

- [Attention Is All You Need - Vaswani et al.](https://arxiv.org/abs/1706.03762)
- [An Image is Worth 16x16 Words: Transformers for Image Recognition - Dosovitskiy et al.](https://arxiv.org/abs/2010.11929)

---

**Próximo Módulo**: Foundation Models para Visão Computacional

## 🎯 Conexão com o Próximo Módulo

Agora que dominamos **Vision Transformers e Mecanismos de Atenção**, estamos preparados para explorar **Foundation Models** para visão computacional.

No **Módulo 9**, veremos como:

### 🔗 **Conexões Diretas:**

1. **Atenção** → **Foundation Models**
   - Vision Transformers usam atenção
   - Foundation Models são baseados em Transformers

2. **Arquiteturas Escaláveis** → **Modelos Massivos**
   - ViT é escalável
   - Foundation Models são modelos massivos

3. **Transfer Learning** → **Zero-shot Learning**
   - ViT é bom para transfer learning
   - Foundation Models permitem zero-shot learning

4. **Interpretabilidade** → **Capacidades Emergentes**
   - ViT é interpretável
   - Foundation Models têm capacidades emergentes

### 🚀 **Evolução Natural:**

- **Atenção** → **Foundation Models**
- **Arquiteturas** → **Modelos Massivos**
- **Transfer Learning** → **Zero-shot Learning**
- **Interpretabilidade** → **Capacidades Emergentes**

Esta transição marca o início da **era dos Foundation Models** em visão computacional!